In [6]:
import network

import pandas as pd
from pyiem.database import get_dbconn

ASOS = get_dbconn("asos")
acursor = ASOS.cursor()

MESOSITE = get_dbconn("mesosite")
mcursor = MESOSITE.cursor()
networks = []
mcursor.execute("""SELECT distinct network from stations where country = 'US' and 
 network ~* 'ASOS' and length(id) < 4""")
for row in mcursor:
    networks.append(row[0])

nt = network.Table(networks)

# nt = network.Table(('IA_ASOS','MO_ASOS','IL_ASOS', 'ND_ASOS', 'AWOS',
#          'WI_ASOS','MN_ASOS', 'SD_ASOS', 'NE_ASOS', 'KS_ASOS',
#          'IN_ASOS','KY_ASOS','OH_ASOS','MI_ASOS'))
# nt = network.Table("IA_ASOS")
rows = []
sz = len(nt.sts.keys())
for i, sid in enumerate(nt.sts.keys()):
    if (
        nt.sts[sid]["archive_begin"] is None
        or nt.sts[sid]["archive_begin"].year > 1973
    ):
        continue
    (print("%04i/%04i " % (i + 1, sz)),)

    # Figure out what our archive coverage is, 41 years would be perfect
    acursor.execute(
        """
    select yr from 
        (SELECT substring(d, 1, 4) as yr, count(*) from 
            (select distinct to_char(valid, 'YYYYmmddHH24') as d from alldata 
            where station = %s and valid > '1973-01-01' and valid < '2014-01-01') as foo 
        GROUP by yr) as foo2 
    WHERE count > (365. * 24 * .9)
    """,
        (sid,),
    )
    if acursor.rowcount < 37:
        print(
            "Skipping %s, due to only have %s/41 years of data"
            % (sid, acursor.rowcount)
        )
        continue

    acursor.execute(
        """
 select sum(case when hr < 8 or hr > 19 then sum else 0 end), sum(sum) from 
 (select substr(d, 9,2)::int as hr, sum(max) from 
 (select to_char((valid at time zone %s ) - '1 minute'::interval, 'YYYYmmddHH24') as d, max(p01i) from alldata 
 where station = %s and valid > '1973-01-01' and valid < '2014-01-01' and
 extract(month from valid) in (5,6,7) GROUP by d) as foo 
 GROUP by hr ORDER by hr ASC) as foo2

    """,
        (nt.sts[sid]["tzname"], sid),
    )

    row = acursor.fetchone()
    if row[0] is None or row[1] < 1:
        continue
    val = row[0] / row[1] * 100.0
    rows.append(
        dict(
            sid=sid,
            lat=nt.sts[sid]["lat"],
            lon=nt.sts[sid]["lon"],
            val=val,
            network=nt.sts[sid]["network"],
        )
    )
    print("%s %s => %.2f" % (sid, nt.sts[sid]["network"], val))

df = pd.DataFrame(rows)

 Populating the interactive namespace from numpy and matplotlib
0002/2195  AGC PA_ASOS => 40.02
0007/2195  Skipping NYC, due to only have 13/41 years of data
0008/2195  AGS GA_ASOS => 37.78
0009/2195  Skipping NYG, due to only have 25/41 years of data
0011/2195  Skipping CEW, due to only have 30/41 years of data
0013/2195  SPS TX_ASOS => 48.62
0015/2195  SPI IL_ASOS => 51.38
0016/2195  Skipping BWD, due to only have 11/41 years of data
0017/2195  BWG KY_ASOS => 51.68
0018/2195  BWI MD_ASOS => 48.16
0023/2195  Skipping 4BL, due to only have 0/41 years of data
0027/2195  Skipping IXD, due to only have 15/41 years of data
0030/2195  Skipping BYH, due to only have 19/41 years of data
0031/2195  Skipping BYI, due to only have 32/41 years of data
0033/2195  Skipping JNW, due to only have 0/41 years of data
0035/2195  Skipping SNA, due to only have 14/41 years of data
0036/2195  Skipping PQI, due to only have 19/41 years of data
0039/2195  Skipping B23, due to only have 0/41 years of data
004

In [29]:
df = pd.DataFrame(rows)
df.set_index("sid", inplace=True)
df["useme"] = True

In [30]:
for sid in ["GRK", "MTC", "IAB"]:
    df["useme"][sid] = False

print(df[df.network == "TX_ASOS"])

          lat        lon  network        val  useme
sid                                                
SPS  33.97860  -98.49280  TX_ASOS  48.618512   True
MAF  31.94250 -102.18917  TX_ASOS  55.114741   True
GRK  31.07000  -97.83000  TX_ASOS   0.000000  False
SAT  29.53369  -98.46978  TX_ASOS  43.699404   True
VCT  28.86140  -96.93030  TX_ASOS  36.691761   True
BRO  25.91461  -97.42313  TX_ASOS  33.942129   True
HOU  29.63747  -95.28245  TX_ASOS  32.453114   True
BPT  29.95000  -94.02000  TX_ASOS  34.248637   True
ELP  31.81111 -106.37583  TX_ASOS  53.966312   True
NGP  27.69263  -97.29109  TX_ASOS  41.965552   True
IAH  29.98443  -95.34144  TX_ASOS  30.092732   True
AMA  35.21936 -101.70592  TX_ASOS  65.368951   True
ACT  31.61797  -97.22830  TX_ASOS  51.155584   True
SJT  31.35167 -100.49500  TX_ASOS  56.661512   True
FTW  32.81978  -97.36244  TX_ASOS  54.131380   True
ABI  32.41131  -99.68189  TX_ASOS  55.941852   True
LBB  33.66364 -101.82278  TX_ASOS  61.982855   True
CRP  27.7730

In [1]:
import numpy as np

from pyiem.plot import MapPlot

df2 = df[df["useme"] == True]

m = MapPlot(
    sector="conus",
    title="Percent of Total Precipitation (JJA) between 8 PM and 8 AM Local Time",
    subtitle="for June, July, and August. based on automated station data from 1973-2013",
)
m.contourf(
    np.array(df2.lon),
    np.array(df2.lat),
    np.array(df2.val),
    np.arange(0, 101, 5),
    units="%",
)
# m.plot_values(df2.lon, df2.lat, df2.val, fmt='%.0f')
m.postprocess(filename="test.png")

NameError: name 'df' is not defined